# Installing libraries and dependencies

In [2]:
# !pip install huggingface_hub
# !pip install datasets
# ! pip install PyDrive
# !pip install python-dotenv
# ! pip install PyPDF2
# ! pip install python-docx
# ! pip install python-pptx
# ! pip install textract
# ! pip install openai
# ! pip install openai==0.28.0


In [3]:
from huggingface_hub import whoami

# Checking if I can access HuggingFace

print(whoami())


{'type': 'user', 'id': '66eedce2801ea45d7a6ce231', 'name': 'ShubhamAgarwalHFace', 'fullname': 'Shubham Agarwal', 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/0345e24b25164c0ae08a2be23554f272.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'GAIA', 'role': 'fineGrained', 'createdAt': '2024-09-21T15:06:03.232Z', 'fineGrained': {'canReadGatedRepos': True, 'global': ['inference.serverless.write', 'discussion.write', 'post.write'], 'scoped': [{'entity': {'_id': '6532270e829e1dc2f293d6b8', 'type': 'dataset'}, 'permissions': ['repo.content.read', 'discussion.write', 'repo.write']}, {'entity': {'_id': '66eedce2801ea45d7a6ce231', 'type': 'user', 'name': 'ShubhamAgarwalHFace'}, 'permissions': ['repo.content.read', 'repo.write', 'inference.endpoints.infer.write', 'inference.endpoints.write', 'user.webhooks.read', 'user.webhooks.write', 'collection.read', 'collection.write', 'discussion.write', 'user.billing.read']}]}}}}


In [4]:
from datasets import load_dataset

#Loading  GAIA dataset.
GAIA_dataset = load_dataset("gaia-benchmark/GAIA", '2023_all')


print(GAIA_dataset)

DatasetDict({
    test: Dataset({
        features: ['task_id', 'Question', 'Level', 'Final answer', 'file_name', 'file_path', 'Annotator Metadata'],
        num_rows: 301
    })
    validation: Dataset({
        features: ['task_id', 'Question', 'Level', 'Final answer', 'file_name', 'file_path', 'Annotator Metadata'],
        num_rows: 165
    })
})


In [5]:
# Snapshot data to local
GAIA_dataset.save_to_disk("/Users/shubhamagarwal/Documents/Northeastern/Semester_3/Big Data/Assignment_1/GAIA_Dataset/")


Saving the dataset (0/1 shards):   0%|          | 0/301 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/165 [00:00<?, ? examples/s]

# Snapshot Dataset from GAIA to local

In [6]:
from huggingface_hub import snapshot_download

# My desired path
my_path = "/Users/shubhamagarwal/Documents/Northeastern/Semester_3/Big Data/Assignment_1/GAIA_Dataset/"

# snapshot dataset to the my desired path
snapshot_download(repo_id="gaia-benchmark/GAIA", repo_type="dataset", local_dir=my_path)

# Reference:https://huggingface.co/docs/huggingface_hub/guides/download#download-an-entire-repository

Fetching 114 files:   0%|          | 0/114 [00:00<?, ?it/s]

'/Users/shubhamagarwal/Documents/Northeastern/Semester_3/Big Data/Assignment_1/GAIA_Dataset'

# Data Conversion to Desired/Workable Format, Relational Format

In [7]:
import pandas as pd
import json


# Loading .jsonl file to  a DataFrame
metadata_df = pd.read_json('/Users/shubhamagarwal/Documents/Northeastern/Semester_3/Big Data/Assignment_1/GAIA_Dataset/2023/validation/metadata.jsonl', lines=True)


In [8]:
metadata_df.head()

,task_id,Question,Level,Final answer,file_name,Annotator Metadata
0,c61d22de-5f6c-4958-a7f6-5e9707bd3466,A paper about AI regulation that was originall...,2,egalitarian,,{'Steps': '1. Go to arxiv.org and navigate to ...
1,17b5a6a3-bc87-42e8-b0fb-6ab0781ef2cc,I’m researching species that became invasive a...,2,34689,,{'Steps': '1. Search the web for “finding nemo...
2,04a04a9b-226c-43fd-b319-d5e89743676f,If we assume all articles published by Nature ...,2,41,,{'Steps': '1. Find how many articles were publ...
3,14569e28-c88c-43e4-8c32-097d35b9a67d,"In Unlambda, what exact charcter or text needs...",2,backtick,,"{'Steps': '1. Searched ""Unlambda syntax"" onlin..."
4,e1fc63a2-da7a-432f-be78-7c4a95598703,If Eliud Kipchoge could maintain his record-ma...,1,17,,{'Steps': '1. Googled Eliud Kipchoge marathon ...


In [9]:
metadata_df.columns

Index(['task_id', 'Question', 'Level', 'Final answer', 'file_name',
       'Annotator Metadata'],
      dtype='object')

# json_normalize to Flatten Nested Fields

In [10]:
# pd.json_normalize: This flattens the Annotator Metadata dictionary into individual columns(Steps, Number of steps, ...)

metadata_df_flattened = pd.json_normalize(metadata_df['Annotator Metadata'])

# flattened Annotator Metadata fields will be combined back with the original DF
metadata_df_combined = pd.concat([metadata_df, metadata_df_flattened], axis=1)


In [11]:
metadata_df_combined.head()

,task_id,Question,Level,Final answer,file_name,Annotator Metadata,Steps,Number of steps,How long did this take?,Tools,Number of tools
0,c61d22de-5f6c-4958-a7f6-5e9707bd3466,A paper about AI regulation that was originall...,2,egalitarian,,{'Steps': '1. Go to arxiv.org and navigate to ...,1. Go to arxiv.org and navigate to the Advance...,12,8 minutes,1. Web browser\n2. Image recognition tools (to...,2
1,17b5a6a3-bc87-42e8-b0fb-6ab0781ef2cc,I’m researching species that became invasive a...,2,34689,,{'Steps': '1. Search the web for “finding nemo...,1. Search the web for “finding nemo main chara...,10,5 minutes,1. Search engine\n2. Web browser,2
2,04a04a9b-226c-43fd-b319-d5e89743676f,If we assume all articles published by Nature ...,2,41,,{'Steps': '1. Find how many articles were publ...,1. Find how many articles were published in Na...,3,5 minutes,1. search engine\n2. calculator,2
3,14569e28-c88c-43e4-8c32-097d35b9a67d,"In Unlambda, what exact charcter or text needs...",2,backtick,,"{'Steps': '1. Searched ""Unlambda syntax"" onlin...","1. Searched ""Unlambda syntax"" online (optional...",6,15 minutes,1. Web browser\n2. Search engine\n3. Unlambda ...,3
4,e1fc63a2-da7a-432f-be78-7c4a95598703,If Eliud Kipchoge could maintain his record-ma...,1,17,,{'Steps': '1. Googled Eliud Kipchoge marathon ...,1. Googled Eliud Kipchoge marathon pace to fin...,4,20 Minutes,1. A web browser.\n2. A search engine.\n3. A c...,3


In [12]:
# metadata_df_combined.to_csv('df_combined.csv')

# Removing Final Answer from Steps

In [13]:
# Removing Final Answer from Steps
def remove_final_answer_from_steps(row):
    steps = row['Steps']
    final_answer = row['Final answer']
    
    # Remove if found
    if final_answer in steps:
        steps = steps.replace(final_answer, '')
    
    return steps

# Using the function created
metadata_df_combined['Steps'] = metadata_df_combined.apply(remove_final_answer_from_steps, axis=1)


In [14]:
metadata_df_combined.head()

,task_id,Question,Level,Final answer,file_name,Annotator Metadata,Steps,Number of steps,How long did this take?,Tools,Number of tools
0,c61d22de-5f6c-4958-a7f6-5e9707bd3466,A paper about AI regulation that was originall...,2,egalitarian,,{'Steps': '1. Go to arxiv.org and navigate to ...,1. Go to arxiv.org and navigate to the Advance...,12,8 minutes,1. Web browser\n2. Image recognition tools (to...,2
1,17b5a6a3-bc87-42e8-b0fb-6ab0781ef2cc,I’m researching species that became invasive a...,2,34689,,{'Steps': '1. Search the web for “finding nemo...,1. Search the web for “finding nemo main chara...,10,5 minutes,1. Search engine\n2. Web browser,2
2,04a04a9b-226c-43fd-b319-d5e89743676f,If we assume all articles published by Nature ...,2,41,,{'Steps': '1. Find how many articles were publ...,1. Find how many articles were published in Na...,3,5 minutes,1. search engine\n2. calculator,2
3,14569e28-c88c-43e4-8c32-097d35b9a67d,"In Unlambda, what exact charcter or text needs...",2,backtick,,"{'Steps': '1. Searched ""Unlambda syntax"" onlin...","1. Searched ""Unlambda syntax"" online (optional...",6,15 minutes,1. Web browser\n2. Search engine\n3. Unlambda ...,3
4,e1fc63a2-da7a-432f-be78-7c4a95598703,If Eliud Kipchoge could maintain his record-ma...,1,17,,{'Steps': '1. Googled Eliud Kipchoge marathon ...,1. Googled Eliud Kipchoge marathon pace to fin...,4,20 Minutes,1. A web browser.\n2. A search engine.\n3. A c...,3


# Updating columns names for naming convention standards throughout the project

In [15]:
new_column_names = ['task_id', 'question', 'level', 'final_answer', 'file_name', 
                    'annotator_metadata', 'steps', 'number_of_steps', 
                    'how_long_did_this_take', 'tools', 'number_of_tools']

# Assigining the new column names to the DataFrame
metadata_df_combined.columns = new_column_names

metadata_df_combined.columns

Index(['task_id', 'question', 'level', 'final_answer', 'file_name',
       'annotator_metadata', 'steps', 'number_of_steps',
       'how_long_did_this_take', 'tools', 'number_of_tools'],
      dtype='object')

In [16]:
#Let's confirm if the data looks good.

metadata_df_combined.head()

,task_id,question,level,final_answer,file_name,annotator_metadata,steps,number_of_steps,how_long_did_this_take,tools,number_of_tools
0,c61d22de-5f6c-4958-a7f6-5e9707bd3466,A paper about AI regulation that was originall...,2,egalitarian,,{'Steps': '1. Go to arxiv.org and navigate to ...,1. Go to arxiv.org and navigate to the Advance...,12,8 minutes,1. Web browser\n2. Image recognition tools (to...,2
1,17b5a6a3-bc87-42e8-b0fb-6ab0781ef2cc,I’m researching species that became invasive a...,2,34689,,{'Steps': '1. Search the web for “finding nemo...,1. Search the web for “finding nemo main chara...,10,5 minutes,1. Search engine\n2. Web browser,2
2,04a04a9b-226c-43fd-b319-d5e89743676f,If we assume all articles published by Nature ...,2,41,,{'Steps': '1. Find how many articles were publ...,1. Find how many articles were published in Na...,3,5 minutes,1. search engine\n2. calculator,2
3,14569e28-c88c-43e4-8c32-097d35b9a67d,"In Unlambda, what exact charcter or text needs...",2,backtick,,"{'Steps': '1. Searched ""Unlambda syntax"" onlin...","1. Searched ""Unlambda syntax"" online (optional...",6,15 minutes,1. Web browser\n2. Search engine\n3. Unlambda ...,3
4,e1fc63a2-da7a-432f-be78-7c4a95598703,If Eliud Kipchoge could maintain his record-ma...,1,17,,{'Steps': '1. Googled Eliud Kipchoge marathon ...,1. Googled Eliud Kipchoge marathon pace to fin...,4,20 Minutes,1. A web browser.\n2. A search engine.\n3. A c...,3


In [17]:
#Saving metadata_df_combined in .csv, local.

metadata_df_combined.to_csv('metadata_df_combined.csv', index=False)

# Mapping GCP object paths to the file name in a new column

In [18]:

gcp_paths = [
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/076c8171-9b3b-49b9-a477-244d2a532826.xlsx",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/1f975693-876d-457b-a649-393859e79bf3.mp3",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/2b3ef98c-cc05-450b-a719-711aee40ac65.mp3",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/32102e3e-d12a-4209-9163-7b3a104efe5d.xlsx",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/366e2f2b-8632-4ef2-81eb-bc3877489217.pdf",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/389793a7-ca17-4e82-81cb-2b3a2391b4b9.txt",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/3da89939-209c-4086-8520-7eb734e6b4ef.xlsx",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/4d0aa727-86b1-406b-9b33-f870dd14a4a5.xlsx",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/4d51c4bf-4b0e-4f3d-897b-3f6687a7d9f2.xlsx",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/54612da3-fd56-4941-80f4-5eb82330de25.xlsx",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/5b2a14e8-6e59-479c-80e3-4696e8980152.jpg",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/5cfb274c-0207-4aa7-9575-6ac0bd95d9b2.xlsx",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/6359a0b1-8f7b-499b-9336-840f9ab90688.png",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/65afbc8a-89ca-4ad5-8d62-355bb401f61d.xlsx",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/67e8878b-5cef-4375-804e-e6291fdbe78a.pdf",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/7bd855d8-463d-4ed5-93ca-5fe35145f733.xlsx",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/7cc4acfa-63fd-4acc-a1a1-e8e529e0a97f.xlsx",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/7dd30055-0198-452e-8c25-f73dbe27dcb8.pdb",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/8d46b8d6-b38a-47ff-ac74-cda14cf2d19b.csv",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/8f80e01c-1296-4371-9486-bb3d68651a60.png",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/9318445f-fe6a-4e1b-acbf-c68228c9906a.png",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/99c9cc74-fdc8-46c6-8f8d-3ce2d3bfeea3.mp3",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/9b54f9d9-35ee-4a14-b62f-d130ea00317f.zip",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/a3fbeb63-0e8c-4a11-bff6-0e3b484c3e9c.pptx",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/b2c257e0-3ad7-4f05-b8e3-d9da973be36e.jpg",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/b7f857e4-d8aa-4387-af2a-0e844df5b9d8.png",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/bec74516-02fc-48dc-b202-55e78d0e17cf.jsonld",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/bfcd99e1-0690-4b53-a85c-0174a8629083.zip",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/c526d8d6-5987-4da9-b24c-83466fa172f3.xlsx",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/cca530fc-4052-43b2-b130-b30968d8aa44.png",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/cca70ce6-1952-45d2-acd4-80c903b0bc49.png",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/cffe0e32-c9a6-4c52-9877-78ceb4aaa9fb.docx",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/d8152ad6-e4d5-4c12-8bb7-8d57dc10c6de.png",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/da52d699-e8d2-4dc5-9191-a2199e0b6a9b.xlsx",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/df6561b2-7ee5-4540-baab-5095f742716a.png",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/e9a2c537-8232-4c3f-85b0-b52de6bcba99.pdf",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/edd4d4f2-1a58-45c4-b038-67337af4e029.xlsx",
    "gs://bigdataia_fall2024_team9_assignment1_bucket/gaia_multimodal/gaia_validation_multimodal/f918266a-b3e0-4914-865d-4faa564f1aef.py"

]




# Creating dictionary mapping file names to their GCP paths
gcp_paths_dict = {path.split("/")[-1]: path for path in gcp_paths}

# Function to get the GCP path if it exists in the dictionary, if not give the original value 
def get_gcp_path(file_name):
    return gcp_paths_dict.get(file_name, None)  # Return None if file_name is not in the dictionary

# Adding  new column gcp_object_path to DF, if paths in the dictionary
metadata_df_combined['gcp_object_path'] = metadata_df_combined['file_name'].apply(get_gcp_path)



In [19]:
metadata_df_combined.head()


,task_id,question,level,final_answer,file_name,annotator_metadata,steps,number_of_steps,how_long_did_this_take,tools,number_of_tools,gcp_object_path
0,c61d22de-5f6c-4958-a7f6-5e9707bd3466,A paper about AI regulation that was originall...,2,egalitarian,,{'Steps': '1. Go to arxiv.org and navigate to ...,1. Go to arxiv.org and navigate to the Advance...,12,8 minutes,1. Web browser\n2. Image recognition tools (to...,2,None
1,17b5a6a3-bc87-42e8-b0fb-6ab0781ef2cc,I’m researching species that became invasive a...,2,34689,,{'Steps': '1. Search the web for “finding nemo...,1. Search the web for “finding nemo main chara...,10,5 minutes,1. Search engine\n2. Web browser,2,None
2,04a04a9b-226c-43fd-b319-d5e89743676f,If we assume all articles published by Nature ...,2,41,,{'Steps': '1. Find how many articles were publ...,1. Find how many articles were published in Na...,3,5 minutes,1. search engine\n2. calculator,2,None
3,14569e28-c88c-43e4-8c32-097d35b9a67d,"In Unlambda, what exact charcter or text needs...",2,backtick,,"{'Steps': '1. Searched ""Unlambda syntax"" onlin...","1. Searched ""Unlambda syntax"" online (optional...",6,15 minutes,1. Web browser\n2. Search engine\n3. Unlambda ...,3,None
4,e1fc63a2-da7a-432f-be78-7c4a95598703,If Eliud Kipchoge could maintain his record-ma...,1,17,,{'Steps': '1. Googled Eliud Kipchoge marathon ...,1. Googled Eliud Kipchoge marathon pace to fin...,4,20 Minutes,1. A web browser.\n2. A search engine.\n3. A c...,3,None


In [20]:
metadata_df_combined.columns

Index(['task_id', 'question', 'level', 'final_answer', 'file_name',
       'annotator_metadata', 'steps', 'number_of_steps',
       'how_long_did_this_take', 'tools', 'number_of_tools',
       'gcp_object_path'],
      dtype='object')

In [21]:
# Moveing gcp_object_path column beside file_name

# List of columns
columns = list(metadata_df_combined.columns)

# position of  file_name and move gcp_object_path beside it
file_name_index = columns.index('file_name')
columns.insert(file_name_index + 1, columns.pop(columns.index('gcp_object_path')))

# updated the column order
gaia_df_gcp_paths = metadata_df_combined[columns]



In [22]:
gaia_df_gcp_paths.head()

,task_id,question,level,final_answer,file_name,gcp_object_path,annotator_metadata,steps,number_of_steps,how_long_did_this_take,tools,number_of_tools
0,c61d22de-5f6c-4958-a7f6-5e9707bd3466,A paper about AI regulation that was originall...,2,egalitarian,,None,{'Steps': '1. Go to arxiv.org and navigate to ...,1. Go to arxiv.org and navigate to the Advance...,12,8 minutes,1. Web browser\n2. Image recognition tools (to...,2
1,17b5a6a3-bc87-42e8-b0fb-6ab0781ef2cc,I’m researching species that became invasive a...,2,34689,,None,{'Steps': '1. Search the web for “finding nemo...,1. Search the web for “finding nemo main chara...,10,5 minutes,1. Search engine\n2. Web browser,2
2,04a04a9b-226c-43fd-b319-d5e89743676f,If we assume all articles published by Nature ...,2,41,,None,{'Steps': '1. Find how many articles were publ...,1. Find how many articles were published in Na...,3,5 minutes,1. search engine\n2. calculator,2
3,14569e28-c88c-43e4-8c32-097d35b9a67d,"In Unlambda, what exact charcter or text needs...",2,backtick,,None,"{'Steps': '1. Searched ""Unlambda syntax"" onlin...","1. Searched ""Unlambda syntax"" online (optional...",6,15 minutes,1. Web browser\n2. Search engine\n3. Unlambda ...,3
4,e1fc63a2-da7a-432f-be78-7c4a95598703,If Eliud Kipchoge could maintain his record-ma...,1,17,,None,{'Steps': '1. Googled Eliud Kipchoge marathon ...,1. Googled Eliud Kipchoge marathon pace to fin...,4,20 Minutes,1. A web browser.\n2. A search engine.\n3. A c...,3


In [23]:
gaia_df_gcp_paths.columns

Index(['task_id', 'question', 'level', 'final_answer', 'file_name',
       'gcp_object_path', 'annotator_metadata', 'steps', 'number_of_steps',
       'how_long_did_this_take', 'tools', 'number_of_tools'],
      dtype='object')

In [24]:
gaia_df_gcp_paths.to_csv('gaia_df.csv', index=False)


In [25]:
gaia_df_gcp_paths.head(25)

,task_id,question,level,final_answer,file_name,gcp_object_path,annotator_metadata,steps,number_of_steps,how_long_did_this_take,tools,number_of_tools
0,c61d22de-5f6c-4958-a7f6-5e9707bd3466,A paper about AI regulation that was originall...,2,egalitarian,,None,{'Steps': '1. Go to arxiv.org and navigate to ...,1. Go to arxiv.org and navigate to the Advance...,12,8 minutes,1. Web browser\n2. Image recognition tools (to...,2
1,17b5a6a3-bc87-42e8-b0fb-6ab0781ef2cc,I’m researching species that became invasive a...,2,34689,,None,{'Steps': '1. Search the web for “finding nemo...,1. Search the web for “finding nemo main chara...,10,5 minutes,1. Search engine\n2. Web browser,2
2,04a04a9b-226c-43fd-b319-d5e89743676f,If we assume all articles published by Nature ...,2,41,,None,{'Steps': '1. Find how many articles were publ...,1. Find how many articles were published in Na...,3,5 minutes,1. search engine\n2. calculator,2
3,14569e28-c88c-43e4-8c32-097d35b9a67d,"In Unlambda, what exact charcter or text needs...",2,backtick,,None,"{'Steps': '1. Searched ""Unlambda syntax"" onlin...","1. Searched ""Unlambda syntax"" online (optional...",6,15 minutes,1. Web browser\n2. Search engine\n3. Unlambda ...,3
4,e1fc63a2-da7a-432f-be78-7c4a95598703,If Eliud Kipchoge could maintain his record-ma...,1,17,,None,{'Steps': '1. Googled Eliud Kipchoge marathon ...,1. Googled Eliud Kipchoge marathon pace to fin...,4,20 Minutes,1. A web browser.\n2. A search engine.\n3. A c...,3
5,32102e3e-d12a-4209-9163-7b3a104efe5d,The attached spreadsheet shows the inventory f...,2,Time-Parking 2: Parallel Universe,32102e3e-d12a-4209-9163-7b3a104efe5d.xlsx,gs://bigdataia_fall2024_team9_assignment1_buck...,{'Steps': '1. Open the attached file. 2. Compa...,1. Open the attached file.\n2. Compare the yea...,3,1 minute,1. Microsoft Excel,1
6,8e867cd7-cff9-4e6c-867a-ff5ddc2550be,How many studio albums were published by Merce...,1,3,,None,{'Steps': '1. I did a search for Mercedes Sosa...,1. I did a search for Mercedes Sosa\n2. I went...,4,5 minutes,1. web browser\n2. google search,2
7,3627a8be-a77f-41bb-b807-7e1bd4c0ebdf,The object in the British Museum's collection ...,2,142,,None,{'Steps': '1. Use search engine to search for ...,"1. Use search engine to search for ""British Mu...",10,12 minutes,1. Web browser\n2. Search engine,2
8,7619a514-5fa8-43ef-9143-83b66a43d7a4,"According to github, when was Regression added...",2,04/15/18,,None,"{'Steps': '1. Searched ""numpy github"" on Googl...","1. Searched ""numpy github"" on Google search.\n...",9,10 minutes,1. Web browser\n2. Search engine,2
9,ec09fa32-d03f-4bf8-84b0-1f16922c3ae4,Here's a fun riddle that I think you'll enjoy....,1,3,,None,{'Steps': 'Step 1: Evaluate the problem statem...,Step 1: Evaluate the problem statement provide...,9,1 minute,None,0


In [26]:
gaia_df = gaia_df_gcp_paths

# Adding file type

In [27]:
# mking a copy, because error-SettingWithCopyWarning
gaia_df_copy = gaia_df.copy()

# Function to extract file extension from file name
def get_file_extension(file_name):
    if pd.isna(file_name):  # Handle NaN values
        return None
    return file_name.split('.')[-1] if '.' in file_name else None

# Using .loc on the DataFrame
gaia_df_copy.loc[:, 'file_type'] = gaia_df_copy['file_name'].apply(get_file_extension)


In [28]:

gaia_df.head(10)

,task_id,question,level,final_answer,file_name,gcp_object_path,annotator_metadata,steps,number_of_steps,how_long_did_this_take,tools,number_of_tools
0,c61d22de-5f6c-4958-a7f6-5e9707bd3466,A paper about AI regulation that was originall...,2,egalitarian,,None,{'Steps': '1. Go to arxiv.org and navigate to ...,1. Go to arxiv.org and navigate to the Advance...,12,8 minutes,1. Web browser\n2. Image recognition tools (to...,2
1,17b5a6a3-bc87-42e8-b0fb-6ab0781ef2cc,I’m researching species that became invasive a...,2,34689,,None,{'Steps': '1. Search the web for “finding nemo...,1. Search the web for “finding nemo main chara...,10,5 minutes,1. Search engine\n2. Web browser,2
2,04a04a9b-226c-43fd-b319-d5e89743676f,If we assume all articles published by Nature ...,2,41,,None,{'Steps': '1. Find how many articles were publ...,1. Find how many articles were published in Na...,3,5 minutes,1. search engine\n2. calculator,2
3,14569e28-c88c-43e4-8c32-097d35b9a67d,"In Unlambda, what exact charcter or text needs...",2,backtick,,None,"{'Steps': '1. Searched ""Unlambda syntax"" onlin...","1. Searched ""Unlambda syntax"" online (optional...",6,15 minutes,1. Web browser\n2. Search engine\n3. Unlambda ...,3
4,e1fc63a2-da7a-432f-be78-7c4a95598703,If Eliud Kipchoge could maintain his record-ma...,1,17,,None,{'Steps': '1. Googled Eliud Kipchoge marathon ...,1. Googled Eliud Kipchoge marathon pace to fin...,4,20 Minutes,1. A web browser.\n2. A search engine.\n3. A c...,3
5,32102e3e-d12a-4209-9163-7b3a104efe5d,The attached spreadsheet shows the inventory f...,2,Time-Parking 2: Parallel Universe,32102e3e-d12a-4209-9163-7b3a104efe5d.xlsx,gs://bigdataia_fall2024_team9_assignment1_buck...,{'Steps': '1. Open the attached file. 2. Compa...,1. Open the attached file.\n2. Compare the yea...,3,1 minute,1. Microsoft Excel,1
6,8e867cd7-cff9-4e6c-867a-ff5ddc2550be,How many studio albums were published by Merce...,1,3,,None,{'Steps': '1. I did a search for Mercedes Sosa...,1. I did a search for Mercedes Sosa\n2. I went...,4,5 minutes,1. web browser\n2. google search,2
7,3627a8be-a77f-41bb-b807-7e1bd4c0ebdf,The object in the British Museum's collection ...,2,142,,None,{'Steps': '1. Use search engine to search for ...,"1. Use search engine to search for ""British Mu...",10,12 minutes,1. Web browser\n2. Search engine,2
8,7619a514-5fa8-43ef-9143-83b66a43d7a4,"According to github, when was Regression added...",2,04/15/18,,None,"{'Steps': '1. Searched ""numpy github"" on Googl...","1. Searched ""numpy github"" on Google search.\n...",9,10 minutes,1. Web browser\n2. Search engine,2
9,ec09fa32-d03f-4bf8-84b0-1f16922c3ae4,Here's a fun riddle that I think you'll enjoy....,1,3,,None,{'Steps': 'Step 1: Evaluate the problem statem...,Step 1: Evaluate the problem statement provide...,9,1 minute,None,0


# Adding column for list of file types supported by OpenAI API

In [29]:
#file types supported by OpenAI API
supported_file_types = ['txt', 'mp3', 'mp4', 'mpeg', 'mpga', 'm4a', 'wav', 'webm', 'png', 'jpg', 'jpeg', 'json']

# to check if the file type is supported
def is_supported(file_type):
    if pd.isna(file_type):  # If the file_type is NaN, return None (or you can return '')
        return None
    elif file_type in supported_file_types:
        return 'supported'
    else:
        return 'not supported'

# Apply to file_type column and create a new supported_by_openai column
gaia_df_copy.loc[:, 'supported_by_openai'] = gaia_df_copy['file_type'].apply(is_supported)


In [30]:
gaia_df_copy.head(25)

,task_id,question,level,final_answer,file_name,gcp_object_path,annotator_metadata,steps,number_of_steps,how_long_did_this_take,tools,number_of_tools,file_type,supported_by_openai
0,c61d22de-5f6c-4958-a7f6-5e9707bd3466,A paper about AI regulation that was originall...,2,egalitarian,,None,{'Steps': '1. Go to arxiv.org and navigate to ...,1. Go to arxiv.org and navigate to the Advance...,12,8 minutes,1. Web browser\n2. Image recognition tools (to...,2,None,None
1,17b5a6a3-bc87-42e8-b0fb-6ab0781ef2cc,I’m researching species that became invasive a...,2,34689,,None,{'Steps': '1. Search the web for “finding nemo...,1. Search the web for “finding nemo main chara...,10,5 minutes,1. Search engine\n2. Web browser,2,None,None
2,04a04a9b-226c-43fd-b319-d5e89743676f,If we assume all articles published by Nature ...,2,41,,None,{'Steps': '1. Find how many articles were publ...,1. Find how many articles were published in Na...,3,5 minutes,1. search engine\n2. calculator,2,None,None
3,14569e28-c88c-43e4-8c32-097d35b9a67d,"In Unlambda, what exact charcter or text needs...",2,backtick,,None,"{'Steps': '1. Searched ""Unlambda syntax"" onlin...","1. Searched ""Unlambda syntax"" online (optional...",6,15 minutes,1. Web browser\n2. Search engine\n3. Unlambda ...,3,None,None
4,e1fc63a2-da7a-432f-be78-7c4a95598703,If Eliud Kipchoge could maintain his record-ma...,1,17,,None,{'Steps': '1. Googled Eliud Kipchoge marathon ...,1. Googled Eliud Kipchoge marathon pace to fin...,4,20 Minutes,1. A web browser.\n2. A search engine.\n3. A c...,3,None,None
5,32102e3e-d12a-4209-9163-7b3a104efe5d,The attached spreadsheet shows the inventory f...,2,Time-Parking 2: Parallel Universe,32102e3e-d12a-4209-9163-7b3a104efe5d.xlsx,gs://bigdataia_fall2024_team9_assignment1_buck...,{'Steps': '1. Open the attached file. 2. Compa...,1. Open the attached file.\n2. Compare the yea...,3,1 minute,1. Microsoft Excel,1,xlsx,not supported
6,8e867cd7-cff9-4e6c-867a-ff5ddc2550be,How many studio albums were published by Merce...,1,3,,None,{'Steps': '1. I did a search for Mercedes Sosa...,1. I did a search for Mercedes Sosa\n2. I went...,4,5 minutes,1. web browser\n2. google search,2,None,None
7,3627a8be-a77f-41bb-b807-7e1bd4c0ebdf,The object in the British Museum's collection ...,2,142,,None,{'Steps': '1. Use search engine to search for ...,"1. Use search engine to search for ""British Mu...",10,12 minutes,1. Web browser\n2. Search engine,2,None,None
8,7619a514-5fa8-43ef-9143-83b66a43d7a4,"According to github, when was Regression added...",2,04/15/18,,None,"{'Steps': '1. Searched ""numpy github"" on Googl...","1. Searched ""numpy github"" on Google search.\n...",9,10 minutes,1. Web browser\n2. Search engine,2,None,None
9,ec09fa32-d03f-4bf8-84b0-1f16922c3ae4,Here's a fun riddle that I think you'll enjoy....,1,3,,None,{'Steps': 'Step 1: Evaluate the problem statem...,Step 1: Evaluate the problem statement provide...,9,1 minute,None,0,None,None


In [31]:
gaia_df_copy.to_csv("gaia_df_copy.csv", index=False)

In [32]:
gaia_df_copy.head(25)

,task_id,question,level,final_answer,file_name,gcp_object_path,annotator_metadata,steps,number_of_steps,how_long_did_this_take,tools,number_of_tools,file_type,supported_by_openai
0,c61d22de-5f6c-4958-a7f6-5e9707bd3466,A paper about AI regulation that was originall...,2,egalitarian,,None,{'Steps': '1. Go to arxiv.org and navigate to ...,1. Go to arxiv.org and navigate to the Advance...,12,8 minutes,1. Web browser\n2. Image recognition tools (to...,2,None,None
1,17b5a6a3-bc87-42e8-b0fb-6ab0781ef2cc,I’m researching species that became invasive a...,2,34689,,None,{'Steps': '1. Search the web for “finding nemo...,1. Search the web for “finding nemo main chara...,10,5 minutes,1. Search engine\n2. Web browser,2,None,None
2,04a04a9b-226c-43fd-b319-d5e89743676f,If we assume all articles published by Nature ...,2,41,,None,{'Steps': '1. Find how many articles were publ...,1. Find how many articles were published in Na...,3,5 minutes,1. search engine\n2. calculator,2,None,None
3,14569e28-c88c-43e4-8c32-097d35b9a67d,"In Unlambda, what exact charcter or text needs...",2,backtick,,None,"{'Steps': '1. Searched ""Unlambda syntax"" onlin...","1. Searched ""Unlambda syntax"" online (optional...",6,15 minutes,1. Web browser\n2. Search engine\n3. Unlambda ...,3,None,None
4,e1fc63a2-da7a-432f-be78-7c4a95598703,If Eliud Kipchoge could maintain his record-ma...,1,17,,None,{'Steps': '1. Googled Eliud Kipchoge marathon ...,1. Googled Eliud Kipchoge marathon pace to fin...,4,20 Minutes,1. A web browser.\n2. A search engine.\n3. A c...,3,None,None
5,32102e3e-d12a-4209-9163-7b3a104efe5d,The attached spreadsheet shows the inventory f...,2,Time-Parking 2: Parallel Universe,32102e3e-d12a-4209-9163-7b3a104efe5d.xlsx,gs://bigdataia_fall2024_team9_assignment1_buck...,{'Steps': '1. Open the attached file. 2. Compa...,1. Open the attached file.\n2. Compare the yea...,3,1 minute,1. Microsoft Excel,1,xlsx,not supported
6,8e867cd7-cff9-4e6c-867a-ff5ddc2550be,How many studio albums were published by Merce...,1,3,,None,{'Steps': '1. I did a search for Mercedes Sosa...,1. I did a search for Mercedes Sosa\n2. I went...,4,5 minutes,1. web browser\n2. google search,2,None,None
7,3627a8be-a77f-41bb-b807-7e1bd4c0ebdf,The object in the British Museum's collection ...,2,142,,None,{'Steps': '1. Use search engine to search for ...,"1. Use search engine to search for ""British Mu...",10,12 minutes,1. Web browser\n2. Search engine,2,None,None
8,7619a514-5fa8-43ef-9143-83b66a43d7a4,"According to github, when was Regression added...",2,04/15/18,,None,"{'Steps': '1. Searched ""numpy github"" on Googl...","1. Searched ""numpy github"" on Google search.\n...",9,10 minutes,1. Web browser\n2. Search engine,2,None,None
9,ec09fa32-d03f-4bf8-84b0-1f16922c3ae4,Here's a fun riddle that I think you'll enjoy....,1,3,,None,{'Steps': 'Step 1: Evaluate the problem statem...,Step 1: Evaluate the problem statement provide...,9,1 minute,None,0,None,None


# Testing openai api with code intrepreter capabilities and files

In [ ]:
import openai
import os
import pandas as pd
import PyPDF2
from docx import Document
import json
from PyPDF2 import PdfReader

# Set your OpenAI API key
openai.api_key = ' I have removed hard coded key'

# Function to transcribe audio using Whisper API
def transcribe_audio(file_path):
    audio_file = open(file_path, "rb")
    transcript = openai.Audio.transcribe("whisper-1", audio_file)
    return transcript['text']

# Function to extract text from a .txt file
def extract_text(file_path):
    with open(file_path, 'r') as file:
        return file.read()

# Function to extract text from a PDF file
def extract_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Function to extract text from a Word (.docx) file
def extract_docx(file_path):
    doc = Document(file_path)
    return '\n'.join([para.text for para in doc.paragraphs])

# Function to extract text from an Excel (.xlsx) file
def extract_excel(file_path):
    df = pd.read_excel(file_path)
    return df.to_string(index=False)

# Function to determine the type of file and extract/transcribe its content
def process_file(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()
    
    if file_extension == '.mp3' or file_extension == '.wav':
        return transcribe_audio(file_path)
    elif file_extension == '.txt':
        return extract_text(file_path)
    elif file_extension == '.pdf':
        return extract_pdf(file_path)
    elif file_extension == '.docx':
        return extract_docx(file_path)
    elif file_extension == '.xlsx':
        return extract_excel(file_path)
    else:
        return "Unsupported file type."

# Function to send a question with file content to OpenAI Chat API
def send_question_with_file(question, file_path):
    # Process the file based on its type
    file_content = process_file(file_path)
    
    # Send the question along with the file content to OpenAI
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"{question}\n\nHere is the file content:\n{file_content}"}
        ]
    )
    
    return response

# Example usage
question =""" """
file_path = ""


# Call the function and get the response
response = send_question_with_file(question, file_path)

# Print the response from OpenAI
print(response['choices'][0]['message']['content'].strip())
